In [34]:
# importing important libraries
import pandas as pd
import numpy as np
import requests # handle requests
import folium # maps plotting lib
from pandas.io.json import json_normalize #to tranforming json file into a pandas dataframe library
from geopy.geocoders import Nominatim  # turn addresses to latitude and longitude

In [45]:
# defining constant variables
CLIENT_ID ='VHNQRNUSLSHZNZJDV2SDR1G4MYOJKXQU2EYHGSETICAZ33LU'
CLIENT_SECRET = 'OJ1P2EXE2VECMS0GGN4ADGR1QKUPI4AZ5IQD1F0BP5WW0YFN'
VERSION = '20200717'
LONDON_LAT = 51.509865
LONDON_LNG = -0.118092
RADIUS = 1000
LIMIT = 200
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d1e0931735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    LONDON_LAT, 
    LONDON_LNG, 
    RADIUS, 
    LIMIT)

In [46]:
# get data from FourSquare API
result = requests.get(url).json()

In [47]:
# Json to dataframe
coffee_shops = result['response']['venues']
nearby_coffee_shops = json_normalize(coffee_shops)
nearby_coffee_shops.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4b8a90cef964a5208b7232e3,Caffè Nero,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1595026498,False,2 Lancaster Pl,51.511114,-0.119181,"[{'label': 'display', 'lat': 51.51111373965925...",158,WC2E 7EA,GB,London,Greater London,United Kingdom,"[2 Lancaster Pl, London, Greater London, WC2E ...",NaN,NaN,NaN
1,5a6dfdcf48b04e70ed5493b5,Hej Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1595026498,False,Strand,51.511265,-0.117505,"[{'label': 'display', 'lat': 51.511265, 'lng':...",161,W C2R,GB,London,Greater London,United Kingdom,"[Strand, London, Greater London, W C2R, United...",NaN,NaN,NaN
2,5d56815d67ac1000087afda4,Orée,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1595026498,False,44 Welington St,51.512625,-0.121031,"[{'label': 'display', 'lat': 51.512625, 'lng':...",368,WC2B 5JD,GB,London,Greater London,United Kingdom,"[44 Welington St (Covent Garden), London, Grea...",Covent Garden,NaN,NaN
3,5b731d28e97dfb002c173f37,WA Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1595026498,False,"5, New Row St",51.511009,-0.126614,"[{'label': 'display', 'lat': 51.511009, 'lng':...",603,WC2N 4LH,GB,London,Greater London,United Kingdom,"[5, New Row St, London, Greater London, WC2N 4...",NaN,NaN,NaN
4,4b840247f964a520e91a31e3,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1595026498,False,"10 Kingsway, Unit B2; St Catherines House",51.513868,-0.117669,"[{'label': 'display', 'lat': 51.513868, 'lng':...",446,WC2B 6LH,GB,London,Greater London,United Kingdom,"[10 Kingsway, Unit B2; St Catherines House, Lo...",NaN,NaN,NaN


In [48]:
# drop unnecessary columns
nearby_coffee_shops.drop(["id","categories","referralId","hasPerk","location.crossStreet","location.labeledLatLngs","location.cc","location.city","location.state","location.country","location.formattedAddress","location.neighborhood",
"location.distance","venuePage.id"],axis=1,inplace = True)
nearby_coffee_shops.head()

,name,location.address,location.lat,location.lng,location.distance,location.postalCode
0,Caffè Nero,2 Lancaster Pl,51.511114,-0.119181,158,WC2E 7EA
1,Hej Coffee,Strand,51.511265,-0.117505,161,W C2R
2,Orée,44 Welington St,51.512625,-0.121031,368,WC2B 5JD
3,WA Cafe,"5, New Row St",51.511009,-0.126614,603,WC2N 4LH
4,Starbucks,"10 Kingsway, Unit B2; St Catherines House",51.513868,-0.117669,446,WC2B 6LH


In [49]:
# clean columns
nearby_coffee_shops.columns = [col.split(".")[-1] for col in nearby_coffee_shops.columns]

In [50]:
# map coffee shops in the dataframe

coffee_shops_map = folium.Map(
    location=[LONDON_LAT,LONDON_LNG],
    zoom_start = 15
)

# add all coffee shops as blue circle markers

for lat, lng, label in zip(nearby_coffee_shops.lat, nearby_coffee_shops.lng, nearby_coffee_shops.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(coffee_shops_map)

In [51]:
coffee_shops_map

In [52]:
# Segmenting and Clustering

from sklearn.cluster import KMeans

k = 4

london_clustering = nearby_coffee_shops.drop(['name','address','distance','postalCode'],1)
kmeans = KMeans(n_clusters=k,random_state=0).fit(london_clustering)
nearby_coffee_shops.insert(0, 'Cluster Labels',kmeans.labels_)
kmeans.labels_

array([1, 1, 1, 3, 1, 0, 1, 3, 2, 1, 1, 0, 0, 0, 0, 0, 3, 1, 1, 1, 1, 3,
       3, 1, 3, 0, 3, 3, 3, 2, 1, 0, 3, 2, 3, 3, 1, 1, 2, 1, 3, 2, 3, 3,
       1, 1, 2, 3, 2, 1])

In [53]:
nearby_coffee_shops.head()

,Cluster Labels,name,address,lat,lng,distance,postalCode
0,1,Caffè Nero,2 Lancaster Pl,51.511114,-0.119181,158,WC2E 7EA
1,1,Hej Coffee,Strand,51.511265,-0.117505,161,W C2R
2,1,Orée,44 Welington St,51.512625,-0.121031,368,WC2B 5JD
3,3,WA Cafe,"5, New Row St",51.511009,-0.126614,603,WC2N 4LH
4,1,Starbucks,"10 Kingsway, Unit B2; St Catherines House",51.513868,-0.117669,446,WC2B 6LH


In [56]:
from matplotlib.pyplot import cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[LONDON_LAT,LONDON_LNG],zoom_start=15)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lng, name, cluster in zip(nearby_coffee_shops['lat'], nearby_coffee_shops['lng'], nearby_coffee_shops['name'], nearby_coffee_shops['Cluster Labels']):
    label = folium.Popup(' Cluster '+str(cluster)+'\n'+ str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters